# Работа со строковыми значениями

Материалы:
* Макрушин С.В. Лекция 8: Работа со строковыми значениям
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
* https://tproger.ru/translations/regular-expression-python/
* https://realpython.com/nltk-nlp-python/

In [219]:
import re
from pathlib import Path
from typing import Iterable
from IPython.display import Markdown, Pretty

import nltk
import pandas as pd

In [203]:

DATA_DIR = Path('data/')
OUTPUT_DIR = Path('output/')

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [14]:
obj = {
    'home_page': 'https://github.com/pypa/sampleproject',
    'keywords': 'sample setuptools development',
    'license': 'MIT'
}
left_max = max([len(i) for i in obj])
for k, v in obj.items():
    print(f"{k:{left_max}} = '{v}'")

home_page = 'https://github.com/pypa/sampleproject'
keywords  = 'sample setuptools development'
license   = 'MIT'


2. Дана строка 'aaa--bbb==ccc__ddd'. Написать регулярное выражение для разбивки строки на список ['aaa','bbb','ccc','ddd'].

In [17]:
s = 'aaa--bbb==ccc__ddd'
pattern = re.compile(r'[a-z]{3}')
pattern.findall(s)

['aaa', 'bbb', 'ccc', 'ddd']

3. Проверить корректность введенного E-mail

In [18]:
email = 'example@email.ru'
pattern = re.compile(r'^\w+@[a-z]+\.[a-z]{2,3}$')
match = pattern.match(email)
match.string

'example@email.ru'

4. Разбейте текст формулировки задачи 1 на слова.

In [25]:
task_text = (
    'Вывести на экран данные из словаря `obj` построчно в виде `k = v`, '
    'задав формат таким образом, чтобы знак равенства оказался на одной и '
    'той же позиции во всех строках. Строковые литералы обернуть в кавычки.'
)
nltk.word_tokenize(task_text)[:10]

['Вывести',
 'на',
 'экран',
 'данные',
 'из',
 'словаря',
 '`',
 'obj',
 '`',
 'построчно']

## Лабораторная работа 8

### Форматирование строк

1.1 Загрузите данные из файла `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в таблицу `recipes` При помощи форматирования строк выведите информацию об id рецепта и кол-ве игредиентов 5 случайных рецептов в виде таблицы следующего вида:

    
    |    id     | n_in  |
    |-------------------|
    |  400894   |  13   |
    |   68588   |   8   |
    |  362081   |   6   |
    |   53408   |  12   |
    |  221203   |   4   |

In [274]:
recipes_df = pd.read_csv(DATA_DIR.joinpath('recipes_sample_with_tags_ingredients.csv'))
recipes_df.head()

,id,name,minutes,contributor_id,submitted,n_steps,description,n_tags,tags,n_ingredients,ingredients
0,44123,george s at the cove black bean soup,90,35193,2002-10-25,11,an original recipe created by chef scott meska...,25,weeknight;time-to-make;course;main-ingredient;...,18,unsalted butter*carrot*onion*celery*broccoli s...
1,67664,healthy for them yogurt popsicles,10,91970,2003-07-26,3,my children and their friends ask for my homem...,31,15-minutes-or-less;time-to-make;course;prepara...,3,milk*frozen juice concentrate*plain yogurt
2,38798,i can t believe it s spinach,30,1533,2002-08-29,5,"these were so go, it surprised even me.",17,30-minutes-or-less;time-to-make;course;main-in...,8,onion*frozen chopped spinach*eggs*garlic powde...
3,35173,italian gut busters,45,22724,2002-07-27,7,my sister-in-law made these for us at a family...,11,60-minutes-or-less;time-to-make;course;prepara...,9,sandwich bun*good seasonings italian salad dre...
4,84797,love is in the air beef fondue sauces,25,4470,2004-02-23,4,i think a fondue is a very romantic casual din...,19,30-minutes-or-less;time-to-make;course;main-in...,12,beef steaks*vegetable oil*spicy mustard*fresh ...


In [275]:
def pprint_df(df: pd.DataFrame, n: int = 5) -> None:
    print(pformat_df(df, n=n))

In [276]:
def pformat_df(df: pd.DataFrame, n: int = 5) -> str:
    header = make_header(df)
    body = make_body(df, n)
    sep = f'|{"-" * (len(header) - 3)}|\n'
    return sep.join([header, body])

In [277]:
def make_header(df: pd.DataFrame) -> str:
    width_max_content = get_max_width_content(df)
    header_template = generate_header_template(width_max_content)
    return header_template.format(*df.columns)

In [278]:
def generate_header_template(width_max_content: Iterable) -> str:
    string_builder = ['{{:^{v}.{v}}} '.format(v=v + 2) for v in width_max_content]
    return f'| {" | ".join(string_builder)} |\n'

In [279]:
def make_body(df: pd.DataFrame, n: int = 5) -> str:
    width_max_content = get_max_width_content(df)
    row_template = generate_row_template(width_max_content)
    string_builder = [row_template.format(*row) for _, row in df.sample(n).astype(str).iterrows()]
    return ''.join(string_builder)

In [280]:
def generate_row_template(width_max_content: Iterable) -> str:
    string_builder = ['{{:>{v}.{v}}}  '.format(v=v + 1) for v in width_max_content]
    return f'| {" | ".join(string_builder)} |\n'

In [281]:
def get_max_width_content(df: pd.DataFrame) -> Iterable:
    return df.max().astype(str).str.len()

In [282]:
id_n_ingredients_df = recipes_df[['id', 'n_ingredients']]
pprint_df(id_n_ingredients_df)

|    id     | n_in  |
|-------------------|
|  232606   |   7   |
|  416860   |  11   |
|   60430   |  12   |
|  375050   |   3   |
|  390305   |  10   |



In [301]:
example = """
|    id     | n_in  |
|-------------------|
|  400894   |  13   |
|   68588   |   8   |
|  362081   |   6   |
|   53408   |  12   |
|  221203   |   4   |
"""
result = pformat_df(id_n_ingredients_df)
Pretty(f'\n{example}\n\n{result}\n')



|    id     | n_in  |
|-------------------|
|  400894   |  13   |
|   68588   |   8   |
|  362081   |   6   |
|   53408   |  12   |
|  221203   |   4   |


|    id     | n_in  |
|-------------------|
|  221429   |   8   |
|  482425   |   4   |
|  484412   |   8   |
|  314908   |   7   |
|   52953   |   9   |



1.2 Напишите функцию `show_info`, которая для рецепта по его `id` создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название"

1. Шаг 1.
2. Шаг 2.
----------
#тэг1 #тэг2
```

    
Данные для создания строки получите из файлов `recipes_sample_with_tags_ingredients.csv`, `steps_sample.xml` (__ЛР4__) и `tags_sample.csv` (__ЛР5__). 
Выведите созданную строку на экран.

## Работа с регулярными выражениями

В задачах данного блока подразумевается, что вы не будете использовать никаких строковые методы (`split`, `startswith` и т.д.). Все задачи необходимо решить при помощи регулярных выражений.

2.1 Посчитайте кол-во отзывов, в которых встречаются числа.

2.2 Найдите все смайлики в отзывах. Смайлик состоит из трех частей: глаза (символ `=` или `:`), нос (символ `-`), губы (символ `)` или `(`). Смайлик может иметь вид "глаза-нос-губы" или "губы-нос-глаза". Нос может отсутствовать.

2.3 Проверьте, что все даты в датасете имеют вид "YYYY-MM-DD". Продемонстрируйте работу вашего решения, приведя пример из датасета и контрпример не из датасета.

2.4 Используя строку-результат задачи 1.2, найдите первое слово каждого шага в рецепте

2.5 Используя регулярные выражения, удалите из описаний все символы, кроме английских букв, цифр и пробелов. Сохраните предобработанные описания в файл `preprocessed_descriptions.csv`, содержащий 2 столбца: `name` и `preprocessed_descriptions`.

### Сегментация текста

3.1 Разбейте предобработанные отзывы из задания 2.5 на предложения, а предложения - на слова (используйте `sent_tokenize` и `word_tokenize` из `nltk`). Каждый отзыв представьте в виде списка списков: внешний список - предложения, вложенные списки - слова в предложении.

`'Предложение номер один. Предложение номер два.' => [['Предложение', 'номер', 'один', '.'], ['Предложение', 'номер', 'два', '.']]`

3.2 Посчитайте кол-во уникальных слов в датасете (без учета регистра).

3.3 Найдите 5 самых длинных (по количеству слов) отзывов в датасете и выведите их в порядке убывания длины.

3.4 Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на любом предложении из отзывов.